In [ ]:
import fsspec
import os
import cudf
import numpy as np
import nvtabular as nvt
from cudf.io.parquet import ParquetWriter

INPUT_PATH = os.environ.get('INPUT_PATH', '/datasets/criteo/crit_orig/')
OUTPUT_PATH = os.environ.get('OUTPUT_PATH', '/raid/criteo/tests/demo_out/')

train_set = [os.path.join(INPUT_PATH, x) for x in os.listdir(INPUT_PATH) if x.startswith("day_") and not x.endswith("parquet")] 
train_set.sort()

cont_names = ["I" + str(x) for x in range(1, 14)]
cat_names = ["C" + str(x) for x in range(1, 27)]
cols = ["label"] + cont_names + cat_names

dtypes = {}
dtypes["label"] = np.int64
for x in cont_names:
    dtypes[x] = np.int64
for x in cat_names:
    dtypes[x] = "hex"
    
for filename in train_set:
    file_path = os.path.join(OUTPUT_PATH, os.path.split(filename)[1].split('.')[0])
    writer = ParquetWriter(f'{file_path}.parquet')
    fs = fsspec.filesystem('file')   
    for gdf in nvt.io.CSVFileReader(filename, fs, names=cols, gpu_memory_frac=0.05, sep='\t', dtypes=dtypes):
        writer.write_table(gdf)
        